# Exploratory Data Analysis (EDA) -- RPHunter -- rechecking

## Imports & Configuration

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import os
import seaborn as sns
import re
from sklearn.preprocessing import MultiLabelBinarizer
from pathlib import Path

%matplotlib inline
sns.set(style="whitegrid")

In [2]:
PATH = os.path.join(Path.cwd().parents[1],'data/external/rphunter')


## Load Dataset

In [3]:
total_df = pd.read_excel(os.path.join(PATH, "Rug-Pull-Incidents.xlsx"), engine='openpyxl', sheet_name="Total")
experiment_df = pd.read_excel(os.path.join(PATH, "Rug-Pull-Incidents.xlsx"), engine='openpyxl', sheet_name="Experiment")
normal_list = os.listdir(os.path.join(PATH, "Normal-Bytecode"))
rug_list = os.listdir(os.path.join(PATH, "Rug-Bytecode"))
normal_bytecode_df = pd.DataFrame({'Address': normal_list})
rug_bytecode_df = pd.DataFrame({'Address': rug_list})
normal_bytecode_df['Address'] = normal_bytecode_df['Address'].apply(lambda x: x.split('.')[0])
rug_bytecode_df['Address'] = rug_bytecode_df['Address'].apply(lambda x: x.split('.')[0])

## Missing Data

In [4]:
# Convert all addresses to lower case for comparison
total_df['Address_lower'] = total_df['Address'].str.lower()
normal_bytecode_df['Address_lower'] = normal_bytecode_df['Address'].str.lower()
rug_bytecode_df['Address_lower'] = rug_bytecode_df['Address'].str.lower()

# Check if Address in total_df exists in normal_bytecode_df or rug_bytecode_df
total_df['in_normal'] = total_df['Address_lower'].isin(normal_bytecode_df['Address_lower'])
total_df['in_rug'] = total_df['Address_lower'].isin(rug_bytecode_df['Address_lower'])
# Keep only rows where the address exists in either normal_bytecode_df or rug_bytecode_df
total_df = total_df[total_df['in_normal'] | total_df['in_rug']].reset_index(drop=True)
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643 entries, 0 to 642
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Project Name           643 non-null    object
 1   Chain                  643 non-null    object
 2   Address                643 non-null    object
 3   Open Source            643 non-null    object
 4   Sale Restrict          206 non-null    object
 5   Variable Manipulation  145 non-null    object
 6   Balance Tamper         195 non-null    object
 7   Source                 642 non-null    object
 8   Address_lower          643 non-null    object
 9   in_normal              643 non-null    bool  
 10  in_rug                 643 non-null    bool  
dtypes: bool(2), object(9)
memory usage: 46.6+ KB


In [5]:
experiment_df['Address_lower'] = experiment_df['Address'].str.lower()
# Check if Address in experiment_df exists in normal_bytecode_df or rug_bytecode_df
experiment_df['in_normal'] = experiment_df['Address_lower'].isin(normal_bytecode_df['Address_lower'])
experiment_df['in_rug'] = experiment_df['Address_lower'].isin(rug_bytecode_df['Address_lower'])
# Keep only rows where the address exists in either normal_bytecode_df or rug_bytecode_df
experiment_df = experiment_df[experiment_df['in_normal'] | experiment_df['in_rug']].reset_index(drop=True)
experiment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Project Name           645 non-null    object
 1   Chain                  645 non-null    object
 2   Address                645 non-null    object
 3   Open Source            645 non-null    object
 4   Sale Restrict          204 non-null    object
 5   Variable Manipulation  145 non-null    object
 6   Balance Tamper         190 non-null    object
 7   Source                 644 non-null    object
 8   Address_lower          645 non-null    object
 9   in_normal              645 non-null    bool  
 10  in_rug                 645 non-null    bool  
dtypes: bool(2), object(9)
memory usage: 46.7+ KB


In [6]:
empty_cond = ((total_df['Balance Tamper'].isna() | (total_df['Balance Tamper'].str.strip() == '')) &
    (total_df['Variable Manipulation'].isna() | (total_df['Variable Manipulation'].str.strip() == '')) &
    (total_df['Sale Restrict'].isna() | (total_df['Sale Restrict'].str.strip() == '')))

empty_or_nan_rows = total_df[empty_cond].shape[0]

print(f"Number of rows with empty or NaN values in 'Balance Tamper', 'Variable Manipulation', and 'Sale Restrict': {empty_or_nan_rows}")

Number of rows with empty or NaN values in 'Balance Tamper', 'Variable Manipulation', and 'Sale Restrict': 221


In [7]:
empty_cond_exp = ((experiment_df['Balance Tamper'].isna() | (experiment_df['Balance Tamper'].str.strip() == '')) &
    (experiment_df['Variable Manipulation'].isna() | (experiment_df['Variable Manipulation'].str.strip() == '')) &
    (experiment_df['Sale Restrict'].isna() | (experiment_df['Sale Restrict'].str.strip() == '')))
empty_or_nan_rows_exp = experiment_df[
    empty_cond_exp
].shape[0]

print(f"Number of rows with empty or NaN values in 'Balance Tamper', 'Variable Manipulation', and 'Sale Restrict': {empty_or_nan_rows_exp}")

Number of rows with empty or NaN values in 'Balance Tamper', 'Variable Manipulation', and 'Sale Restrict': 229


In [8]:
# remove empty_or_nan_rows from total_df
total_df = total_df[~empty_cond]

total_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 422 entries, 0 to 421
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Project Name           422 non-null    object
 1   Chain                  422 non-null    object
 2   Address                422 non-null    object
 3   Open Source            422 non-null    object
 4   Sale Restrict          206 non-null    object
 5   Variable Manipulation  145 non-null    object
 6   Balance Tamper         195 non-null    object
 7   Source                 422 non-null    object
 8   Address_lower          422 non-null    object
 9   in_normal              422 non-null    bool  
 10  in_rug                 422 non-null    bool  
dtypes: bool(2), object(9)
memory usage: 33.8+ KB


In [9]:
total_df.drop(columns=["Address_lower", "in_normal", "in_rug"])

,Project Name,Chain,Address,Open Source,Sale Restrict,Variable Manipulation,Balance Tamper,Source
0,GMETA,BSC,0X93023F1D3525E273F291B6F76D2F5027A39BF302,Yes,NaN,Modifiable Tax Rate,Hidden Mint/Burn,https://twitter.com/BeosinAlert/status/1681240...
1,PokémonFi,BSC,0X2753DCE37A7EDB052A77832039BCC9AA49AD8B25,Yes,Address Restrict,NaN,NaN,https://twitter.com/CertiKAlert/status/1562555...
2,Sudorare,ETH,0X5404EFAFDD8CC30053069DF2A1B0C4BA881B3E1E,Yes,NaN,NaN,Hidden Mint/Burn,https://x.com/PeckShieldAlert/status/156196749...
3,DRAC Network,ETH,0X10F6F2B97F3AB29583D9D38BABF2994DF7220C21,Yes,NaN,Modifiable Tax Rate,Hidden Mint/Burn,https://twitter.com/PeckShieldAlert/status/155...
4,DHE,BSC,0X11CBC781DADAAD13FC3A361772C80B1C027820AF,Yes,Address Restrict,NaN,NaN,https://twitter.com/CertiKAlert/status/1539031...
...,...,...,...,...,...,...,...,...
417,Wealthfront,ETH,0XA837EFD4500966A45148073F57FA855205597612,Yes,NaN,NaN,Hidden Mint/Burn,https://de.fi/rekt-database/Wealthfront
418,ANT1GOON,ETH,0X61DB2FB090D890EA9FC09DD837FD47CF2F92EE26,Yes,Address Restrict,Modifiable External Call,NaN,https://de.fi/rekt-database/ANT1GOON
419,Exbase Finance,ETH,0X15C5339954A1016C77732EA626CC714B0D37DD6A,Yes,NaN,Modifiable Tax Rate,NaN,https://de.fi/rekt-database/Exbase Finance
420,Xdef finance,ETH,0X5166D4CE79B9BF7DF477DA110C560CE3045AA889,Yes,Address Restrict,Modifiable Tax Rate,NaN,https://de.fi/rekt-database/Xdef finance


In [10]:
experiment_df = experiment_df[~empty_cond_exp]
experiment_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 416 entries, 0 to 415
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Project Name           416 non-null    object
 1   Chain                  416 non-null    object
 2   Address                416 non-null    object
 3   Open Source            416 non-null    object
 4   Sale Restrict          204 non-null    object
 5   Variable Manipulation  145 non-null    object
 6   Balance Tamper         190 non-null    object
 7   Source                 416 non-null    object
 8   Address_lower          416 non-null    object
 9   in_normal              416 non-null    bool  
 10  in_rug                 416 non-null    bool  
dtypes: bool(2), object(9)
memory usage: 33.3+ KB


In [11]:
total_df.drop(columns=["Address_lower", "in_normal", "in_rug"])

,Project Name,Chain,Address,Open Source,Sale Restrict,Variable Manipulation,Balance Tamper,Source
0,GMETA,BSC,0X93023F1D3525E273F291B6F76D2F5027A39BF302,Yes,NaN,Modifiable Tax Rate,Hidden Mint/Burn,https://twitter.com/BeosinAlert/status/1681240...
1,PokémonFi,BSC,0X2753DCE37A7EDB052A77832039BCC9AA49AD8B25,Yes,Address Restrict,NaN,NaN,https://twitter.com/CertiKAlert/status/1562555...
2,Sudorare,ETH,0X5404EFAFDD8CC30053069DF2A1B0C4BA881B3E1E,Yes,NaN,NaN,Hidden Mint/Burn,https://x.com/PeckShieldAlert/status/156196749...
3,DRAC Network,ETH,0X10F6F2B97F3AB29583D9D38BABF2994DF7220C21,Yes,NaN,Modifiable Tax Rate,Hidden Mint/Burn,https://twitter.com/PeckShieldAlert/status/155...
4,DHE,BSC,0X11CBC781DADAAD13FC3A361772C80B1C027820AF,Yes,Address Restrict,NaN,NaN,https://twitter.com/CertiKAlert/status/1539031...
...,...,...,...,...,...,...,...,...
417,Wealthfront,ETH,0XA837EFD4500966A45148073F57FA855205597612,Yes,NaN,NaN,Hidden Mint/Burn,https://de.fi/rekt-database/Wealthfront
418,ANT1GOON,ETH,0X61DB2FB090D890EA9FC09DD837FD47CF2F92EE26,Yes,Address Restrict,Modifiable External Call,NaN,https://de.fi/rekt-database/ANT1GOON
419,Exbase Finance,ETH,0X15C5339954A1016C77732EA626CC714B0D37DD6A,Yes,NaN,Modifiable Tax Rate,NaN,https://de.fi/rekt-database/Exbase Finance
420,Xdef finance,ETH,0X5166D4CE79B9BF7DF477DA110C560CE3045AA889,Yes,Address Restrict,Modifiable Tax Rate,NaN,https://de.fi/rekt-database/Xdef finance


In [12]:
total_df['Balance Tamper'].value_counts()

Balance Tamper
Hidden Mint/Burn                                  136
Hidden Balance Modification                        47
Hidden Mint/Burn,Hidden Balance Modification        5
\nHidden Mint/Burn                                  4
Hidden Mint/Burn茂录聦Hidden Balance Modification      1
\n\nHidden Mint/Burn                                1
\n                                                  1
Name: count, dtype: int64

In [13]:
experiment_df['Balance Tamper'].value_counts()

Balance Tamper
Hidden Mint/Burn                               132
Hidden Balance Modify                           46
Hidden Mint/Burn,Hidden Balance Modify           5
\nHidden Mint/Burn                               4
Hidden Mint/Burn脙炉脗录脗聦Hidden Balance Modify      1
\n\nHidden Mint/Burn                             1
\n                                               1
Name: count, dtype: int64

In [14]:
def clean_balance_tamper(value):
    if pd.isna(value):
        return ''

    # Strip whitespace and linebreaks
    value = value.strip()

    # Replace multiple linebreaks or strange characters with a comma
    value = re.sub(r'[\n\r]+', ',', value)
    value = re.sub(r'[茂录聦]', ',', value)  # Remove strange unicode artifacts
    value = re.sub(r'[脙炉脗录脗聦]', ',', value)  # Remove strange unicode artifacts

    # Correct known typos
    value = value.replace("Hidden Mint/Eurn", "Hidden Mint/Burn")
    value = value.replace("Hidden Balance Modify", "Hidden Balance Modification")

    # Split by comma and normalize each part
    parts = [p.strip() for p in value.split(',') if p.strip()]

    # Deduplicate and sort for consistency
    parts = sorted(set(parts))

    return list(set(parts))

# Apply to both dataframes
total_df['Balance Tamper'] = total_df['Balance Tamper'].apply(clean_balance_tamper)
experiment_df['Balance Tamper'] = experiment_df['Balance Tamper'].apply(clean_balance_tamper)

# Optional: show cleaned value counts
mlb = MultiLabelBinarizer()
tamper_encoded = pd.DataFrame(mlb.fit_transform(total_df['Balance Tamper']),
                              columns=mlb.classes_,
                              index=total_df.index)

total_df = pd.concat([total_df.drop(columns=['Balance Tamper']), tamper_encoded], axis=1)

amper_encoded = pd.DataFrame(mlb.fit_transform(experiment_df['Balance Tamper']),
                              columns=mlb.classes_,
                              index=experiment_df.index)
experiment_df = pd.concat([experiment_df.drop(columns=['Balance Tamper']), tamper_encoded], axis=1)

In [15]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 422 entries, 0 to 421
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Project Name                 422 non-null    object
 1   Chain                        422 non-null    object
 2   Address                      422 non-null    object
 3   Open Source                  422 non-null    object
 4   Sale Restrict                206 non-null    object
 5   Variable Manipulation        145 non-null    object
 6   Source                       422 non-null    object
 7   Address_lower                422 non-null    object
 8   in_normal                    422 non-null    bool  
 9   in_rug                       422 non-null    bool  
 10  Hidden Balance Modification  422 non-null    int64 
 11  Hidden Mint/Burn             422 non-null    int64 
dtypes: bool(2), int64(2), object(8)
memory usage: 37.1+ KB


In [16]:
experiment_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 422 entries, 0 to 421
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Project Name                 416 non-null    object
 1   Chain                        416 non-null    object
 2   Address                      416 non-null    object
 3   Open Source                  416 non-null    object
 4   Sale Restrict                204 non-null    object
 5   Variable Manipulation        145 non-null    object
 6   Source                       416 non-null    object
 7   Address_lower                416 non-null    object
 8   in_normal                    416 non-null    object
 9   in_rug                       416 non-null    object
 10  Hidden Balance Modification  422 non-null    int64 
 11  Hidden Mint/Burn             422 non-null    int64 
dtypes: int64(2), object(10)
memory usage: 42.9+ KB


In [17]:
total_df['Sale Restrict'].value_counts()

Sale Restrict
Address Restrict                                           93
Amount Restrict                                            62
Address Restrict,Amount Restrict                           27
Address Restrict,TimeStamp Restrict                         7
\nAddress Restrict,Amount Restrict                          4
Address Restrict,Amount Restrict,TimeStamp Restrict         3
TimeStamp Restrict                                          2
Modifiable External Call                                    1
Amount Restrict,TimeStamp Restrict                          1
Address Restrict,Address Restrict*3                         1
Address Restrict                                            1
\nAddress Restrict                                          1
\n\nAddress Restrict,Amount Restrict,TimeStamp Restrict     1
\nAmount Restrict                                           1
TimeStanp Restrict                                          1
Name: count, dtype: int64

In [18]:
experiment_df['Sale Restrict'].value_counts()

Sale Restrict
Address Restrict                                           92
Amount Restrict                                            62
Address Restrict,Amount Restrict                           27
Address Restrict,TimeStamp Restrict                         7
\nAddress Restrict,Amount Restrict                          4
Address Restrict,Amount Restrict,TimeStamp Restrict         3
TimeStamp Restrict                                          2
Modifiable External Call                                    1
Amount Restrict,TimeStamp Restrict                          1
Address Restrict                                            1
Address Restrict,Address Restrict*3                         1
\nAddress Restrict                                          1
\n\nAddress Restrict,Amount Restrict,TimeStamp Restrict     1
\nAmount Restrict                                           1
Name: count, dtype: int64

In [19]:
def extract_sale_restrict_labels(value):
    if pd.isna(value) or not str(value).strip():
        return []

    value = re.sub(r'[\n\r]+', ',', value)

    # Fix known typos
    value = value.replace("TimeStanp Restrict", "TimeStamp Restrict")

    # Remove suffix like "*3" (e.g. Address Restrict*3)
    value = re.sub(r'\*[\d]+', '', value)

    # Split and clean
    parts = [p.strip() for p in value.split(',') if p.strip()]

    # Normalize duplicates
    parts = sorted(set(parts))
    return parts

# Apply label extraction
total_df['Sale Restrict'] = total_df['Sale Restrict'].apply(extract_sale_restrict_labels)
experiment_df['Sale Restrict'] = experiment_df['Sale Restrict'].apply(extract_sale_restrict_labels)

# Binarize
mlb_sale = MultiLabelBinarizer()

sale_encoded = pd.DataFrame(mlb_sale.fit_transform(total_df['Sale Restrict']),
                            columns=mlb_sale.classes_,
                            index=total_df.index)

# Drop original and concat
total_df = pd.concat([total_df.drop(columns=['Sale Restrict']), sale_encoded], axis=1)

# Do same for experiment_df
exp_sale_encoded = pd.DataFrame(mlb_sale.transform(experiment_df['Sale Restrict']),
                                columns=mlb_sale.classes_,
                                index=experiment_df.index)

experiment_df = pd.concat([experiment_df.drop(columns=['Sale Restrict']), exp_sale_encoded], axis=1)


In [20]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 422 entries, 0 to 421
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Project Name                 422 non-null    object
 1   Chain                        422 non-null    object
 2   Address                      422 non-null    object
 3   Open Source                  422 non-null    object
 4   Variable Manipulation        145 non-null    object
 5   Source                       422 non-null    object
 6   Address_lower                422 non-null    object
 7   in_normal                    422 non-null    bool  
 8   in_rug                       422 non-null    bool  
 9   Hidden Balance Modification  422 non-null    int64 
 10  Hidden Mint/Burn             422 non-null    int64 
 11  Address Restrict             422 non-null    int64 
 12  Amount Restrict              422 non-null    int64 
 13  Modifiable External Call     422 non-nul

In [21]:
experiment_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 422 entries, 0 to 421
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Project Name                 416 non-null    object
 1   Chain                        416 non-null    object
 2   Address                      416 non-null    object
 3   Open Source                  416 non-null    object
 4   Variable Manipulation        145 non-null    object
 5   Source                       416 non-null    object
 6   Address_lower                416 non-null    object
 7   in_normal                    416 non-null    object
 8   in_rug                       416 non-null    object
 9   Hidden Balance Modification  422 non-null    int64 
 10  Hidden Mint/Burn             422 non-null    int64 
 11  Address Restrict             422 non-null    int64 
 12  Amount Restrict              422 non-null    int64 
 13  Modifiable External Call     422 non-nul

In [22]:
total_df['Variable Manipulation'].value_counts()

Variable Manipulation
Modifiable Tax Rate                                   70
Modifiable External Call                              42
Modifiable Tax Address                                17
Modifiable Tax Rate,Modifiable Tax Address             6
Modifibale Tax Rate                                    4
\nModifiable Tax Rate                                  2
Modifuable Tax Rate                                    1
Modifibale Tax Rate,Modifiable Tax Address             1
Modifiable Tax Address,Hidden Balance Modification     1
\n                                                     1
Name: count, dtype: int64

In [23]:
experiment_df['Variable Manipulation'].value_counts()

Variable Manipulation
Modifiable Tax Rate                             69
Modifiable External Call                        43
Modifiable Tax Address                          17
Modifiable Tax Rate,Modifiable Tax Address       6
Modifibale Tax Rate                              4
\nModifiable Tax Rate                            2
Modifuable Tax Rate                              1
Modifibale Tax Rate,Modifiable Tax Address       1
Modifiable Tax Address,Hidden Balance Modify     1
\n                                               1
Name: count, dtype: int64

In [24]:
def extract_variable_manipulation_labels(value):
    if pd.isna(value) or not str(value).strip():
        return []

    value = re.sub(r'[\n\r]+', ',', value)

    # Correct typos
    value = value.replace("Modifibale Tax Rate", "Modifiable Tax Rate")
    value = value.replace("Modifuable Tax Rate", "Modifiable Tax Rate")
    value = value.replace("Hidden Balance Modify", "")  # strip unrelated label

    # Split and clean
    parts = [p.strip() for p in value.split(',') if p.strip()]

    # Deduplicate
    parts = sorted(set(parts))
    return parts

# Apply function
total_df['Variable Manipulation'] = total_df['Variable Manipulation'].apply(extract_variable_manipulation_labels)
experiment_df['Variable Manipulation'] = experiment_df['Variable Manipulation'].apply(extract_variable_manipulation_labels)

# Binarize
mlb_var = MultiLabelBinarizer()
var_encoded = pd.DataFrame(mlb_var.fit_transform(total_df['Variable Manipulation']),
                           columns=mlb_var.classes_,
                           index=total_df.index)

# Merge and drop
total_df = pd.concat([total_df.drop(columns=['Variable Manipulation']), var_encoded], axis=1)

# Repeat for experiment_df
exp_var_encoded = pd.DataFrame(mlb_var.transform(experiment_df['Variable Manipulation']),
                               columns=mlb_var.classes_,
                               index=experiment_df.index)

experiment_df = pd.concat([experiment_df.drop(columns=['Variable Manipulation']), exp_var_encoded], axis=1)

In [25]:
SAVE_PATH = os.path.join(Path.cwd().parents[1], "data/interim/rphunter")
total_df.to_csv(os.path.join(SAVE_PATH, 'total.csv'), index=False)
experiment_df.to_csv(os.path.join(SAVE_PATH, 'experiment.csv'), index=False)